## Homework

The goal of this homework is to create a simple training pipeline, use mlflow to track experiments and register best model, and a modern data workflow orchestration tool such as Mage or Prefect or others as listed in the course material under '03-orchestration'.

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), the **Yellow** taxi data for March, 2023. 

## Question 1. Select the Tool

You can use the same tool you used when completing the module,
or choose a different one for your homework.

What's the name of the orchestrator you chose? 

Answer: Prefect

## Question 2. Version

What's the version of the orchestrator? 

In [1]:
!prefect version

Version:              3.4.25
API version:          0.8.4
Python version:       3.9.24
Git commit:           8a37e7b1
Built:                Thu, Oct 23, 2025 07:58 PM
OS/Arch:              darwin/arm64
Profile:              local
Server type:          ephemeral
Pydantic version:     2.12.3
Server:
  Database:           sqlite
  SQLite version:     3.50.2


## Question 3. Creating a pipeline

Let's read the March 2023 Yellow taxi trips data.

How many records did we load? 

- 3,003,766
- 3,203,766
- 3,403,766
- 3,603,766

In [9]:
import pandas as pd
import numpy as np
import mlflow
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [4]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet"
df = pd.read_parquet(url)
print(f"✅ Loaded {len(df):,} records")

✅ Loaded 3,403,766 records


## Question 4. Data preparation

Let's continue with pipeline creation.

We will use the same logic for preparing the data we used previously. 

This is what we used (adjusted for yellow dataset):

```python
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df
```

Let's apply to the data we loaded in question 3. 

What's the size of the result? 

- 2,903,766
- 3,103,766
- 3,316,216 
- 3,503,766

In [5]:
def prepare_data(df):
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df['duration'].dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()
    df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)
    return df

In [6]:
df = prepare_data(df)
print(f"✅ Cleaned data. Remaining records: {len(df):,}")

✅ Cleaned data. Remaining records: 3,316,216


## Question 5. Train a model

We will now train a linear regression model using the same code as in homework 1.

* Fit a dict vectorizer.
* Train a linear regression with default parameters.
* Use pick up and drop off locations separately, don't create a combination feature.

Let's now use it in the pipeline. We will need to create another transformation block, and return both the dict vectorizer and the model.

What's the intercept of the model? 

Hint: print the `intercept_` field in the code block

- 21.77
- 24.77
- 27.77
- 31.77

In [10]:
categorical = ['PULocationID', 'DOLocationID']
numerical = []

dv = DictVectorizer() # turns dictionary to a vector
train_dicts = df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_train)

RMSE = root_mean_squared_error(y_train, y_pred)
RMSE

8.158681362238536

In [11]:
print("Intercept:", linear.intercept_)

Intercept: 24.778964270944773


In [16]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("homework3-nyc-taxi")

with mlflow.start_run():
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    rmse = root_mean_squared_error(y_train, y_pred)
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="models",
        input_example=X_train[:1],
        signature=mlflow.models.signature.infer_signature(X_train, y_pred)
    )

2025/10/29 19:28:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


## Question 6. Register the model 

The model is trained, so let's save it with MLFlow.

Find the logged model, and find MLModel file. What's the size of the model? (`model_size_bytes` field):

* 14,534
* 9,534
* 4,534
* 1,534